# **NeuralDB**
NeuralDB 是 ThirdAI 开发的一款对 CPU 友好且可微调的检索引擎。

### **初始化**
有两种初始化方法：
- 从零开始：基础模型
- 从Checkpoint：加载先前保存的模型

对于以下所有初始化方法，如果设置了 `THIRDAI_KEY` 环境变量，则可以省略 `thirdai_key` 参数。

ThirdAI API 密钥可在 https://www.thirdai.com/try-bolt/ 获取

In [ ]:
from langchain_community.retrievers import NeuralDBRetriever

# From scratch
retriever = NeuralDBRetriever.from_scratch(thirdai_key="your-thirdai-key")

# From checkpoint
retriever = NeuralDBRetriever.from_checkpoint(
    # Path to a NeuralDB checkpoint. For example, if you call
    # retriever.save("/path/to/checkpoint.ndb") in one script, then you can
    # call NeuralDBRetriever.from_checkpoint("/path/to/checkpoint.ndb") in
    # another script to load the saved model.
    checkpoint="/path/to/checkpoint.ndb",
    thirdai_key="your-thirdai-key",
)

### **插入文档源**

In [ ]:
retriever.insert(
    # If you have PDF, DOCX, or CSV files, you can directly pass the paths to the documents
    sources=["/path/to/doc.pdf", "/path/to/doc.docx", "/path/to/doc.csv"],
    # When True this means that the underlying model in the NeuralDB will
    # undergo unsupervised pretraining on the inserted files. Defaults to True.
    train=True,
    # Much faster insertion with a slight drop in performance. Defaults to True.
    fast_mode=True,
)

from thirdai import neural_db as ndb

retriever.insert(
    # If you have files in other formats, or prefer to configure how
    # your files are parsed, then you can pass in NeuralDB document objects
    # like this.
    sources=[
        ndb.PDF(
            "/path/to/doc.pdf",
            version="v2",
            chunk_size=100,
            metadata={"published": 2022},
        ),
        ndb.Unstructured("/path/to/deck.pptx"),
    ]
)

### **检索文档**
要查询检索器，您可以使用标准的 LangChain 检索器方法 `get_relevant_documents`，它会返回一个 LangChain Document 对象列表。每个文档对象代表索引文件中一段文本。例如，它可能包含索引的 PDF 文件中的一个段落。除了文本之外，文档的元数据字段还包含文档 ID、文档来源（来自哪个文件）以及文档分数等信息。

In [ ]:
# This returns a list of LangChain Document objects
documents = retriever.invoke("query", top_k=10)

### **微调**
NeuralDBRetriever 可以根据用户行为和领域特定知识进行微调。可以通过两种方式进行微调：
1. 关联：检索器将源短语与目标短语关联起来。当检索器看到源短语时，它还会考虑与目标短语相关的结果。
2. 点赞：检索器会提高特定查询文档的分数。当您想根据用户行为微调检索器时，此功能非常有用。例如，如果用户搜索“汽车是如何制造的”并且喜欢返回的 id 为 52 的文档，那么我们可以为查询“汽车是如何制造的”点赞 id 为 52 的文档。

In [ ]:
retriever.associate(source="source phrase", target="target phrase")
retriever.associate_batch(
    [
        ("source phrase 1", "target phrase 1"),
        ("source phrase 2", "target phrase 2"),
    ]
)

retriever.upvote(query="how is a car manufactured", document_id=52)
retriever.upvote_batch(
    [
        ("query 1", 52),
        ("query 2", 20),
    ]
)